In [1]:
"""
***********************
-----------------------
         SETUP
-----------------------
***********************
"""

'\n***********************\n-----------------------\n         SETUP\n-----------------------\n***********************\n'

In [2]:
# Imports

# Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Models and sklearn
import joblib
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Feature Importance
from sklearn import decomposition
from sklearn.ensemble import StackingRegressor

#Cross Fold Validation
from sklearn.model_selection import cross_val_score

#joblib
import joblib

In [3]:
# get the path of data
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!!other users, pls input your path!!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
train_data_path = '/Users/rakeshmehta/Desktop/AI-Collision-Algo/Data/train_data.csv'
test_data_path = '/Users/rakeshmehta/Desktop/AI-Collision-Algo/Data/test_data.csv'
print(train_data_path)
print(test_data_path)

# put the training and testing data into a DataFrame
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# clean up data by removing column of UNKNOWN from the training and testing DataFrames
train_data = train_data.drop(columns = ['c_object_type'])
test_data = test_data.drop(columns = ['c_object_type'])

print(train_data.head(10))
print(test_data.head(10))

/Users/rakeshmehta/Desktop/AI-Collision-Algo/Data/train_data.csv
/Users/rakeshmehta/Desktop/AI-Collision-Algo/Data/test_data.csv
   event_id  time_to_tca  mission_id       risk  max_risk_estimate  \
0         0     1.566798           5 -10.204955          -7.834756   
1         0     1.207494           5 -10.355758          -7.848937   
2         0     0.952193           5 -10.345631          -7.847406   
3         0     0.579669           5 -10.337809          -7.845880   
4         0     0.257806           5 -10.391260          -7.852942   
5         1     6.530455           5  -7.561299          -7.254301   
6         1     5.561646           5  -9.315693          -7.468904   
7         1     5.226504           5  -7.422508          -7.051001   
8         1     3.570013           5  -9.248105          -7.327533   
9         2     6.983474           2 -10.816161          -6.601713   

   max_risk_scaling  miss_distance  relative_speed  relative_position_r  \
0          8.602101      

In [4]:
# set up the Features and Lables
# training
X_train = train_data.drop(columns=['max_risk_scaling', 'max_risk_estimate'])
y_train = train_data['risk'].values
# testing
X_test = test_data.drop(columns=['max_risk_scaling', 'max_risk_estimate'])
y_test = test_data['risk'].values

# verify the shape of the matrices of data
print("--------------------------------------------")
print("                 Features")
print("--------------------------------------------")
print("Training Features Shape: ", X_train.shape)
print("Testing Features Shape: ", X_test.shape)
print("--------------------------------------------")
print("                  Labels")
print("--------------------------------------------")
print("Training Labels Shape: ", y_train.shape)
print("Testing Labels Shape: ", y_test.shape)

--------------------------------------------
                 Features
--------------------------------------------
Training Features Shape:  (162634, 100)
Testing Features Shape:  (24484, 100)
--------------------------------------------
                  Labels
--------------------------------------------
Training Labels Shape:  (162634,)
Testing Labels Shape:  (24484,)


In [5]:
# create an imputer object with strategy (e.g. mean, median, mode) to get rid of NaN values
imputer = SimpleImputer(strategy='mean')

# Apply imputation to the Data
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

# Print out shape again to verify X and y are same size
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(162634, 100)
(162634,)
(24484, 100)
(24484,)


In [6]:
"""
***********************
-----------------------
      REGRESSIONS
-----------------------
***********************
"""

'\n***********************\n-----------------------\n      REGRESSIONS\n-----------------------\n***********************\n'

In [7]:
# creating and training the linear regression model
l_reg = linear_model.LinearRegression()
l_reg.fit(X_train, y_train)

LinearRegression()

In [8]:
# creating and training the bayesian ridge regression model
b_ridge = linear_model.BayesianRidge()
b_ridge.fit(X_train, y_train)

BayesianRidge()

In [9]:
# creating and training the lasso regression model
lasso_model = Lasso(alpha=0.1)  # You can adjust the regularization parameter 'alpha' as needed
lasso_model.fit(X_train, y_train)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.625e+04, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [10]:
# creating and training the ElasticNet regression model
elastic_net = linear_model.ElasticNet()
elastic_net.fit(X_train, y_train)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.210e+05, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(


ElasticNet()

In [11]:
print("--------------------------------------------")
print("             LINEAR REGRESSION")
print("--------------------------------------------")

# Predict probabilities on the test set
l_reg_predictions = l_reg.predict(X_test)

# Evaluate the model using mean squared error (MSE)
l_reg_mse = mean_squared_error(y_test, l_reg_predictions)
print("Mean Squared Error:", l_reg_mse)

# Evaluate the model using r^2 value
print("r^2: ", l_reg.score(X_test, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(l_reg_predictions)

print("--------------------------------------------")
print("               BAYESIAN RIDGE")
print("--------------------------------------------")

# Predict probabilities on the test set
b_ridge_predictions = b_ridge.predict(X_test)

# Evaluate the model using mean squared error (MSE)
b_ridge_mse = mean_squared_error(y_test, b_ridge_predictions)
print("Mean Squared Error:", b_ridge_mse)

# Evaluate the model using r^2 value
print("r^2: ", b_ridge.score(X_test, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(b_ridge_predictions)

print("--------------------------------------------")
print("                   LASSO")
print("--------------------------------------------")

# Predict probabilities on the test set
lasso_predictions = lasso_model.predict(X_test)

# Evaluate the model using mean squared error (MSE)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
print("Mean Squared Error:", lasso_mse)

# Evaluate the model using r^2 value
print("r^2: ", lasso_model.score(X_test, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(lasso_predictions)

print("--------------------------------------------")
print("               ELASTIC NET")
print("--------------------------------------------")

# Predict probabilities on the test set
elastic_predictions = elastic_net.predict(X_test)

# Evaluate the model using mean squared error (MSE)
elastic_mse = mean_squared_error(y_test, elastic_predictions)
print("Mean Squared Error:", elastic_mse)

# Evaluate the model using r^2 value
print("r^2: ", elastic_net.score(X_test, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(elastic_predictions)

--------------------------------------------
             LINEAR REGRESSION
--------------------------------------------
Mean Squared Error: 104.98112715210861
r^2:  -0.08277063863319833
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[-19.50425754 -19.50425754 -19.50425754 ... -19.50425754 -19.50425754
 -19.50425754]
--------------------------------------------
               BAYESIAN RIDGE
--------------------------------------------
Mean Squared Error: 1125.2879940203347
r^2:  -10.6061699182013
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[-25.5384235 -25.5384235 -25.5384235 ... -25.5384235 -25.5384235
 -25.5384235]
--------------------------------------------
                   LASSO
--------------------------------------------
Mean Squared Error: 0.0001235159145735766
r^2:  0.9999987260623949
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[ -7.3063194   -7.29179511  -7.3

In [12]:
# save the models in joblib
joblib.dump(l_reg, 'linear_regression.joblib')
joblib.dump(b_ridge, 'bayesian_ridge.joblib')
joblib.dump(lasso_model, 'lasso_model.joblib')
joblib.dump(elastic_net, 'elastic_net.joblib')

['elastic_net.joblib']

In [13]:
"""
***********************
-----------------------
  FEATURE IMPORTANCE
-----------------------
***********************
"""

'\n***********************\n-----------------------\n  FEATURE IMPORTANCE\n-----------------------\n***********************\n'

In [14]:
# implement pca
pca = decomposition.PCA(n_components=90) # change the number of components as needed
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# convert back to a dataframe
X_train_pca = pd.DataFrame(X_train_pca)

print(X_train_pca.shape)
print(X_train_pca.head())

(162634, 90)
             0             1             2             3             4   \
0 -8.873973e+44 -1.215205e+45 -2.856293e+28 -1.142517e+29  1.785183e+28   
1 -8.873973e+44 -1.215205e+45 -2.856293e+28 -1.142517e+29  1.785183e+28   
2 -8.873973e+44 -1.215205e+45 -2.856293e+28 -1.142517e+29  1.785183e+28   
3 -8.873973e+44 -1.215205e+45 -2.856293e+28 -1.142517e+29  1.785183e+28   
4 -8.873973e+44 -1.215205e+45 -2.856293e+28 -1.142517e+29  1.785183e+28   

             5             6             7             8             9   ...  \
0 -1.218693e+29 -1.271769e+28 -2.001290e+29 -2.160912e+29  1.142632e+29  ...   
1 -1.218693e+29 -1.271769e+28 -2.001290e+29 -2.160912e+29  1.142632e+29  ...   
2 -1.218693e+29 -1.271769e+28 -2.001290e+29 -2.160912e+29  1.142632e+29  ...   
3 -1.218693e+29 -1.271769e+28 -2.001290e+29 -2.160912e+29  1.142632e+29  ...   
4 -1.218693e+29 -1.271769e+28 -2.001290e+29 -2.160912e+29  1.142632e+29  ...   

             80            81            82            

In [15]:
# Retraining all the models after PCA

In [16]:
# Linear Regression
l_reg_pca = linear_model.LinearRegression()
l_reg_pca.fit(X_train_pca, y_train)

LinearRegression()

In [17]:
# Bayesian Ridge
b_ridge_pca = linear_model.BayesianRidge()
b_ridge_pca.fit(X_train_pca , y_train)

BayesianRidge()

In [18]:
# Lasso
lasso_pca = Lasso(alpha=0.1)  # You can adjust the regularization parameter 'alpha' as needed
lasso_pca.fit(X_train_pca, y_train)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.084e+06, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [19]:
# ElasticNet
elastic_net_pca = linear_model.ElasticNet(alpha=0.1)
elastic_net_pca.fit(X_train_pca, y_train)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.084e+06, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.1)

In [20]:
print("--------------------------------------------")
print("             LINEAR REGRESSION")
print("--------------------------------------------")

# Predict probabilities on the test set
l_reg_predictions = l_reg_pca.predict(X_test_pca)

# Evaluate the model using mean squared error (MSE)
l_reg_mse = mean_squared_error(y_test, l_reg_predictions)
print("Mean Squared Error:", l_reg_mse)

# Evaluate the model using r^2 value
print("r^2: ", l_reg_pca.score(X_test_pca, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(l_reg_predictions)

print("--------------------------------------------")
print("               BAYESIAN RIDGE")
print("--------------------------------------------")

# Predict probabilities on the test set
b_ridge_predictions = b_ridge_pca.predict(X_test_pca)

# Evaluate the model using mean squared error (MSE)
b_ridge_mse = mean_squared_error(y_test, b_ridge_predictions)
print("Mean Squared Error:", b_ridge_mse)

# Evaluate the model using r^2 value
print("r^2: ", b_ridge_pca.score(X_test_pca, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(b_ridge_predictions)

print("--------------------------------------------")
print("                   LASSO")
print("--------------------------------------------")

# Predict probabilities on the test set
lasso_predictions = lasso_pca.predict(X_test_pca)

# Evaluate the model using mean squared error (MSE)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
print("Mean Squared Error:", lasso_mse)

# Evaluate the model using r^2 value
print("r^2: ", lasso_pca.score(X_test_pca, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(lasso_predictions)

print("--------------------------------------------")
print("               ELASTIC NET")
print("--------------------------------------------")

# Predict probabilities on the test set
elastic_predictions = elastic_net_pca.predict(X_test_pca)

# Evaluate the model using mean squared error (MSE)
elastic_mse = mean_squared_error(y_test, elastic_predictions)
print("Mean Squared Error:", elastic_mse)

# Evaluate the model using r^2 value
print("r^2: ", elastic_net_pca.score(X_test_pca, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(elastic_predictions)

--------------------------------------------
             LINEAR REGRESSION
--------------------------------------------
Mean Squared Error: 104.98112715210857
r^2:  -0.08277063863319811
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[-19.50425754 -19.50425754 -19.50425754 ... -19.50425754 -19.50425754
 -19.50425754]
--------------------------------------------
               BAYESIAN RIDGE
--------------------------------------------
Mean Squared Error: 370.04138434071217
r^2:  -2.8165902473381665
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[-21.87719405 -21.87719405 -21.87719405 ... -21.87719405 -21.87719405
 -21.87719405]
--------------------------------------------
                   LASSO
--------------------------------------------
Mean Squared Error: 104.98112715210864
r^2:  -0.08277063863319878
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[-19.50425754 -19.5042575

In [45]:
#create a dictionary of our models
estimators=[('lasso', lasso_model), ('elasticnet', elastic_net)]

# set up the ensemble, using a gradient boosting regressor in this case
final_estimator = GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, 
                                            max_features=1, random_state=42)
reg = StackingRegressor(estimators=estimators, final_estimator=final_estimator)
reg.fit(X_train, y_train)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.625e+04, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.210e+05, tolerance: 1.630e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

StackingRegressor(estimators=[('lasso', Lasso(alpha=0.1)),
                              ('elasticnet', ElasticNet())],
                  final_estimator=GradientBoostingRegressor(max_features=1,
                                                            min_samples_leaf=25,
                                                            n_estimators=25,
                                                            random_state=42,
                                                            subsample=0.5))

In [46]:
print("--------------------------------------------")
print("              GRADIENT BOOSTING")
print("--------------------------------------------")

# Predict probabilities on the test set
reg_predictions = reg.predict(X_test)

# Evaluate the model using mean squared error (MSE)
reg_mse = mean_squared_error(y_test, reg_predictions)
print("Mean Squared Error:", reg_mse)

# Evaluate the model using r^2 value
print("r^2: ", reg.score(X_test, y_test))

# print out predictions and actual values to compare visually
print(y_test)
print(reg_predictions)

--------------------------------------------
              GRADIENT BOOSTING
--------------------------------------------
Mean Squared Error: 0.610068935705113
r^2:  0.9937077763494899
[ -7.2969667   -7.28249593  -7.31605287 ... -25.43687504 -25.5380515
 -25.12796033]
[ -8.17947859  -8.17947859  -8.17947859 ... -25.04166825 -25.04166825
 -24.58810217]


In [23]:
# Run Cross-Fold Validation

cv_scores = cross_val_score(reg, X_train, y_train, cv=5)
print(cv_scores)

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.300e+04, tolerance: 1.302e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.678e+04, tolerance: 1.302e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.743e+04, tolerance: 1.044e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.300e+04, tolerance: 1.307e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.743e+04, tolerance: 1.044e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.743e+04, tolerance: 1.041e+03
  model = cd_fast.enet_coordinate_descent(
/Users/rakeshmehta/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

[0.99451022 0.99459981 0.99454358 0.99452101 0.99454414]


In [47]:
# save the ensembled model in joblib
joblib.dump(reg, 'ensemble.joblib')

['ensemble.joblib']